In [ ]:
import pandas as pd

df = pd.read_parquet("data/filtered_funcs.parquet")

print(df[0])

  language                                               body
0   Python  # Python\n# Check is the PDF valid \nis_valid_...
1   Python  # Python\n# Returns date\n# \n# >>> expiration...
2   Python  # Python\n# Returns dict like {'y': ..., 'm': ...
3   Python  # Python\n# Handle lazy JSON - to fix expectin...
4   Python  # Python\n# Return the collection which is the...


In [4]:
code_snippets = df['body'].tolist()

In [ ]:
from tokenizers import ByteLevelBPETokenizer

tokenizer = ByteLevelBPETokenizer()

tokenizer.train_from_iterator(code_snippets, special_tokens=[
    "<s>", "<pad>", "</s>", "<unk>", "<mask>", "<func>", "</func>"
])

tokenizer.save_model("./tokenizer")


Exception: No such file or directory (os error 2)

In [7]:
tokenizer.save_model("tokenizer")

['tokenizer/vocab.json', 'tokenizer/merges.txt']

In [9]:
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
import pandas as pd
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("tokenizer")
tokenizer.pad_token = "<pad>"
tokenizer.bos_token = "<func>"
tokenizer.eos_token = "</func>"

def add_special_tokens(example):
    example['body'] = f"{tokenizer.bos_token} {example['body']} {tokenizer.eos_token}"
    return example


def tokenize_function(examples):
    return tokenizer(
    examples['body'], 
    return_tensors="np", 
    padding="max_length",
    max_length=1024
)


df = pd.read_parquet(f"data/filtered_funcs.parquet")
    
train, valid = train_test_split(df, train_size=0.8, test_size=0.2, random_state=42)

ds = DatasetDict({
    'train': Dataset.from_pandas(train),
    'eval': Dataset.from_pandas(valid)}
)

ds = ds.map(add_special_tokens)
tokenized_ds = ds.map(tokenize_function, batched=True)
print(tokenized_ds)

tokenized_ds["train"].to_parquet("data/filtered_funcs_tokenized_scratch_train.parquet")
tokenized_ds["eval"].to_parquet("data/filtered_funcs_tokenized_scratch_eval.parquet")

/home/diekhoff/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/5121227 [00:00<?, ? examples/s]

Map:   0%|          | 0/1280307 [00:00<?, ? examples/s]

Map:   0%|          | 0/5121227 [00:00<?, ? examples/s]

Map:   0%|          | 0/1280307 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['language', 'body', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 5121227
    })
    eval: Dataset({
        features: ['language', 'body', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 1280307
    })
})


Creating parquet from Arrow format:   0%|          | 0/5122 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1281 [00:00<?, ?ba/s]

7165872537